In [6]:
# Cellule 1 : Import des librairies et définition des constantes

import requests
import base64

# Renseignez ici vos identifiants Spotify
CLIENT_ID = "2a4cf4aebe7946a0ab1782a95c6a72ae"
CLIENT_SECRET = "165c9df45fef4cba90962018ad7337f9"  # À remplacer par le vrai secret

# Identifiant de la playlist (extrait d'une URL de type https://open.spotify.com/playlist/XXXXX)
PLAYLIST_ID = "37i9dQZF1DXcBWIGoYBM5M"


In [7]:
# Cellule 2 : Définition de la fonction pour obtenir l'access token

def get_access_token(client_id, client_secret):
    """
    Récupère un token d'accès via le Client Credentials Flow.
    """
    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization": "Basic " + base64.b64encode(
            f"{client_id}:{client_secret}".encode("utf-8")
        ).decode("utf-8")
    }
    data = {
        "grant_type": "client_credentials"
    }

    response = requests.post(url, headers=headers, data=data)
    if response.status_code != 200:
        # Affiche l'erreur si nécessaire
        print(f"Erreur lors de la récupération du token : {response.text}")
        return None

    token_info = response.json()
    return token_info.get("access_token")

In [8]:
# Cellule 3 : Appel de get_access_token pour tester la récupération du token

access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)

if access_token:
    print("Token récupéré avec succès :", access_token[:50], "...")
else:
    print("Impossible de récupérer un token")


Token récupéré avec succès : BQCPL8LFt9GSy1NjTSgZBFzjCamkb9rQRYePLqNpKfQm7uMIkl ...


In [9]:
# Cellule 4 : Définition de la fonction pour récupérer les infos d'une playlist

def get_playlist_info(access_token, playlist_id):
    """
    Récupère les informations de la playlist (titre, description, pistes...) via la Web API Spotify.
    """
    url = f"https://api.spotify.com/v1/playlists/{playlist_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)
    
    if response.status_code != 200:
        print(f"Erreur lors de la récupération de la playlist : {response.text}")
        return None
    
    return response.json()


In [10]:
# Cellule 5 : Récupération et affichage des informations de la playlist

if access_token:
    playlist_data = get_playlist_info(access_token, PLAYLIST_ID)
    if playlist_data:
        # Affichage d'informations de base
        print("Nom de la playlist :", playlist_data.get("name", "Inconnu"))
        print("Description :", playlist_data.get("description", "Aucune description"))
        tracks_info = playlist_data.get("tracks", {})
        print("Nombre de morceaux :", tracks_info.get("total", "N/A"))

        # Affichage des titres (limite à la première 'page')
        items = tracks_info.get("items", [])
        for idx, item in enumerate(items, 1):
            track = item.get("track")
            if track:
                track_name = track["name"]
                artists = ", ".join(artist["name"] for artist in track["artists"])
                print(f"{idx}. {track_name} - {artists}")
    else:
        print("Impossible de récupérer les données de playlist")
else:
    print("Pas de token disponible, la playlist ne peut pas être récupérée.")


Erreur lors de la récupération de la playlist : {"error": {"status": 404, "message": "Resource not found" } }
Impossible de récupérer les données de playlist


In [14]:
import streamlit as st
import requests
import base64

# --------------------------------------------------
# 1) Configuration et identifiants Spotify
# --------------------------------------------------
CLIENT_ID = "2a4cf4aebe7946a0ab1782a95c6a72ae"
CLIENT_SECRET = "165c9df45fef4cba90962018ad7337f9"  # Évitez de laisser ça en clair !

# --------------------------------------------------
# 2) Fonctions utilitaires pour l'API Spotify
# --------------------------------------------------
def get_access_token(client_id, client_secret):
    """
    Récupère un token d'accès via le Client Credentials Flow.
    Permet d'accéder aux métadonnées publiques (artists, albums, tracks...).
    """
    url = "https://accounts.spotify.com/api/token"
    auth_string = f"{client_id}:{client_secret}"
    auth_bytes = auth_string.encode("utf-8")
    auth_base64 = base64.b64encode(auth_bytes).decode("utf-8")

    headers = {
        "Authorization": f"Basic {auth_base64}",
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {
        "grant_type": "client_credentials"
    }
    response = requests.post(url, headers=headers, data=data)

    if response.status_code == 200:
        token_info = response.json()
        return token_info["access_token"]
    else:
        st.error(f"Erreur token : {response.status_code} - {response.text}")
        return None

def search_artist(access_token, artist_name):
    """
    Recherche un artiste par nom et renvoie l'ID du premier résultat.
    """
    url = "https://api.spotify.com/v1/search"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "q": artist_name,
        "type": "artist",
        "limit": 1
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        artists = data.get("artists", {}).get("items", [])
        if len(artists) > 0:
            return artists[0]  # on retourne l'objet complet (et pas juste l'ID)
        else:
            return None
    else:
        st.error(f"Erreur lors de la recherche d'artiste : {response.status_code} - {response.text}")
        return None

def get_artist_info(access_token, artist_id):
    """
    Récupère les informations générales d'un artiste (followers, genres, popularité...).
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return response.json()
    else:
        st.error(f"Erreur infos artiste : {response.status_code} - {response.text}")
        return None

def get_artist_albums(access_token, artist_id, limit=20):
    """
    Récupère la liste des albums (et singles, compilations) de l'artiste.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}/albums"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "include_groups": "album,single,compilation",
        "limit": limit
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json().get("items", [])
    else:
        st.error(f"Erreur albums artiste : {response.status_code} - {response.text}")
        return []

def get_artist_top_tracks(access_token, artist_id, market="FR"):
    """
    Récupère le top tracks de l'artiste (souvent 10 titres) pour un marché donné.
    """
    url = f"https://api.spotify.com/v1/artists/{artist_id}/top-tracks"
    headers = {
        "Authorization": f"Bearer {access_token}"
    }
    params = {
        "market": market
    }
    response = requests.get(url, headers=headers, params=params)

    if response.status_code == 200:
        return response.json().get("tracks", [])
    else:
        st.error(f"Erreur top tracks artiste : {response.status_code} - {response.text}")
        return []

# --------------------------------------------------
# 3) Application Streamlit
# --------------------------------------------------
def main():
    st.title("Informations sur un artiste Spotify")

    st.write(
        """
        Entrez le nom d'un artiste ci-dessous pour récupérer plusieurs 
        informations (popularité, albums, top tracks...).
        """
    )

    # Saisir le nom de l'artiste
    artist_name = st.text_input("Nom de l'artiste :", "Booba")

    # Choix d'un marché (pays) pour les top tracks
    market_choice = st.selectbox("Choisir le marché pour les top tracks :", ["FR", "US", "ES", "GB"])

    # Bouton de recherche
    if st.button("Rechercher"):
        with st.spinner("Récupération du token..."):
            access_token = get_access_token(CLIENT_ID, CLIENT_SECRET)
        
        if not access_token:
            st.error("Impossible d'obtenir un token, vérifiez vos identifiants.")
            return
        
        with st.spinner("Recherche de l'artiste..."):
            artist_data = search_artist(access_token, artist_name)
        
        if artist_data is None:
            st.error(f"Aucun artiste trouvé pour '{artist_name}'.")
            return
        
        # artist_data est un dictionnaire contenant l'ID et d'autres infos
        artist_id = artist_data["id"]
        
        st.subheader(f"Résultat pour : {artist_data['name']} (ID = {artist_id})")

        # Infos générales
        with st.spinner("Récupération des informations de l'artiste..."):
            info = get_artist_info(access_token, artist_id)
            if info:
                st.markdown("**Informations générales :**")
                st.write(f"Nom : {info.get('name', 'N/A')}")
                st.write(f"Popularité : {info.get('popularity', 'N/A')}")
                st.write(f"Nombre de followers : {info.get('followers', {}).get('total', 'N/A')}")
                st.write(f"Genres : {', '.join(info.get('genres', [])) or 'N/A'}")
                
                images = info.get("images", [])
                if images:
                    # On affiche la plus grande image
                    st.image(images[0]['url'], caption=f"Image de l'artiste {info['name']}")
        
        # Albums
        with st.spinner("Récupération des albums/singles..."):
            albums = get_artist_albums(access_token, artist_id, limit=20)
            st.markdown("**Quelques albums/singles :**")
            if albums:
                for alb in albums[:10]:  # on affiche par exemple 10 max
                    alb_name = alb.get("name")
                    alb_release = alb.get("release_date")
                    alb_type = alb.get("album_type", "unknown")
                    st.write(f"- ({alb_type}) {alb_name} | Sorti le {alb_release}")
            else:
                st.write("Aucun album trouvé (ou erreur).")
        
        # Top tracks
        with st.spinner("Récupération du top tracks..."):
            top_tracks = get_artist_top_tracks(access_token, artist_id, market=market_choice)
            st.markdown(f"**Top tracks (marché = {market_choice}) :**")
            if top_tracks:
                for i, track in enumerate(top_tracks, 1):
                    track_name = track.get("name")
                    track_pop = track.get("popularity")
                    st.write(f"{i}. {track_name} (popularité : {track_pop})")
            else:
                st.write("Aucun top track trouvé (ou erreur).")

if __name__ == "__main__":
    main()


2025-01-13 13:41:39.454 
  command:

    streamlit run c:\Users\asse-\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-01-13 13:41:39.455 Session state does not function when running a script without `streamlit run`
